In [16]:
import pandas as pd
import numpy as np

In [17]:
postseason_stats = pd.read_csv('postseason_stats.csv')
postseason_stats['Team'] = postseason_stats['Team'].replace({'WSN':'WSH', 'ARI': 'AZ', 'KCR': 'KC', 'TBR': 'TB', 'SFG': 'SF', 'SDP': 'SD', 'CHW': 'CWS'})

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [18]:
rs_all_teams_primary_pitch_metrics = None
for year in [2017, 2018, 2019, 2021, 2022, 2023, 2024]:
    print(year)
    raw = pd.read_csv(f'raw_{year}.csv')
    raw['pitcher'] = raw['pitcher'].astype(str)
    rs = raw.loc[raw['game_type'] == 'R', :]
    rs = rs.loc[rs['pitch_name'] != "Pitch Out", :]
    ps = raw.loc[(raw['game_type'] == 'F') | (raw['game_type'] == 'D') | (raw['game_type'] == 'L') | (raw['game_type'] == 'W'), :]
    ps_pitchers_by_team = {key:None for key in np.unique(np.concatenate([ps['home_team'].unique(), ps['away_team'].unique()]))}
    for team in ps_pitchers_by_team.keys():
        ps_pitchers_by_team[team] = ps.loc[((ps['home_team'] == team) & (ps['inning_topbot'] == 'Top')) | ((ps['away_team'] == team) & (ps['inning_topbot'] == 'Bot')), 'pitcher'].unique()
    for team in ps_pitchers_by_team.keys():
        rs_team = rs.loc[np.isin(rs['pitcher'], ps_pitchers_by_team[team]), ['pitcher', 'p_throws', 'pitch_name', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
        rs_team_grouped = rs_team.groupby(['pitcher'])
        rs_team_primary_pitches = rs_team_grouped['pitch_name'].agg(pd.Series.mode).to_frame()
        rs_team_primary_pitch_metrics = None
        for pitcher in ps_pitchers_by_team[team]:
            try:
                rs_primary_pitch = rs_team.loc[(rs_team['pitcher'] == pitcher) & (rs_team['pitch_name'] == rs_team_primary_pitches.loc[pitcher, :].values[0]), ['release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
                rs_primary_pitch_metrics = rs_primary_pitch.agg('mean').to_frame().transpose()
                if rs_team_primary_pitch_metrics is None:
                    rs_team_primary_pitch_metrics = rs_primary_pitch_metrics
                else:
                    rs_team_primary_pitch_metrics = pd.concat([rs_team_primary_pitch_metrics, rs_primary_pitch_metrics])
            except:
                continue
        rs_teamwide_primary_pitch_metrics = rs_team_primary_pitch_metrics.agg('std').to_frame().transpose()
        rs_teamwide_primary_pitch_metrics['Season'] = year
        rs_teamwide_primary_pitch_metrics['Team'] = team
        rs_teamwide_primary_pitch_metrics['n_primary_pitches'] = rs_team_primary_pitches['pitch_name'].nunique()
        rs_teamwide_primary_pitch_metrics['n_pitchers_used'] = ps_pitchers_by_team[team].shape[0]
        if rs_all_teams_primary_pitch_metrics is None:
            rs_all_teams_primary_pitch_metrics = rs_teamwide_primary_pitch_metrics
        else:
            rs_all_teams_primary_pitch_metrics = pd.concat([rs_all_teams_primary_pitch_metrics, rs_teamwide_primary_pitch_metrics])
        print(team)
data = pd.merge(postseason_stats, rs_all_teams_primary_pitch_metrics, how = 'left', on = ['Season', 'Team'])

2017
AZ
BOS
CHC
CLE
COL
HOU
LAD
MIN
NYY
WSH
2018
ATL
BOS
CHC
CLE
COL
HOU
LAD
MIL
NYY
OAK
2019
ATL
HOU
LAD
MIL
MIN
NYY
OAK
STL
TB
WSH
2021
ATL
BOS
CWS
HOU
LAD
MIL
NYY
SF
STL
TB
2022
ATL
CLE
HOU
LAD
NYM
NYY
PHI
SD
SEA
STL
TB
TOR
2023
ATL
AZ
BAL
HOU
LAD
MIA
MIL
MIN
PHI
TB
TEX
TOR
2024


C:\Users\John Olsen\AppData\Local\Temp\ipykernel_18816\2822689505.py:4: DtypeWarning: Columns (0,76,85,86) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv(f'raw_{year}.csv')


ATL
BAL
CLE
DET
HOU
KC
LAD
MIL
NYM
NYY
PHI
SD


In [19]:
data

,Season,Team,TG,W,L,G,IP,K9,BB9,HR9,BABIP,LOB,GB,HRFB,ERA,FIP,xFIP,release_pos_x,release_pos_z,release_pos_y,release_speed,pfx_x,pfx_z,release_spin_rate,spin_axis,n_primary_pitches,n_pitchers_used
0,2023,PHI,13,8,5,64,114.1,9.37,2.44,0.63,0.276,0.826,0.471,0.072,2.20,2.98,3.67,2.035822,0.488370,0.404634,5.697546,0.976048,0.762360,298.286215,63.246107,5,12
1,2022,HOU,13,11,2,62,126.0,11.43,3.07,0.93,0.232,0.835,0.419,0.108,2.29,3.08,3.15,0.775311,0.541237,0.231211,3.581276,0.755793,0.508551,200.010911,45.707217,3,12
2,2018,MIL,10,6,4,58,93.2,10.76,2.98,0.48,0.282,0.813,0.468,0.069,2.31,2.62,3.19,2.001886,0.395097,0.511292,5.366292,0.745656,0.634608,146.132456,53.381281,5,13
3,2021,ATL,16,11,5,80,140.0,9.96,3.21,0.84,0.293,0.781,0.420,0.096,3.28,3.41,3.91,1.837290,0.473407,0.282884,4.609040,0.674457,0.796224,257.061669,64.774175,6,15
4,2019,WSH,17,12,5,66,153.0,10.24,3.35,1.35,0.275,0.817,0.363,0.135,3.47,4.09,4.35,2.056008,0.427102,0.209599,5.302726,0.659403,0.686615,216.817254,52.090199,5,12
5,2018,BOS,14,11,3,73,134.0,8.73,3.83,1.07,0.239,0.797,0.440,0.117,3.29,4.21,4.34,2.193541,0.429033,0.389136,2.726804,0.918784,0.210677,128.116002,38.192974,2,11
6,2022,PHI,17,11,6,85,150.0,9.84,2.40,1.02,0.262,0.767,0.479,0.118,3.24,3.38,3.33,1.835090,0.391337,0.539885,5.519691,0.953488,0.524841,225.525233,43.986861,6,14
7,2017,NYY,13,7,6,48,115.1,9.29,3.59,0.86,0.230,0.762,0.447,0.106,2.97,3.63,4.00,1.613960,0.303774,0.367187,6.510280,0.704613,0.554072,144.383003,46.269031,4,11
8,2019,NYY,9,5,4,55,81.2,9.81,4.19,1.32,0.228,0.852,0.394,0.150,2.87,4.45,4.49,2.081585,0.354467,0.476906,6.395621,0.884216,0.596053,321.452129,51.158954,6,13
9,2023,AZ,17,10,7,89,152.1,8.63,3.66,1.24,0.267,0.772,0.459,0.137,3.54,4.45,4.32,1.978993,0.775794,0.294590,3.474280,0.847363,0.651450,224.561525,57.416247,4,13


In [20]:
data.to_csv('teamwide_dataset.csv', index = False)

In [7]:
from pybaseball import playerid_reverse_lookup

In [10]:
playerid_reverse_lookup([519222])

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,romine,austin,519222,romia002,rominau01,5491,2011.0,2022.0


Austin Romine (catcher) threw an inning in the 2018 playoffs when the Yankees were getting blown out by Boston

In [5]:
postseason_stats_2017 = postseason_stats.loc[postseason_stats['Season'] == 2017, :]

In [6]:
postseason_stats_2017

,Season,Team,TG,W,L,G,IP,K9,BB9,HR9,BABIP,LOB,GB,HRFB,ERA,FIP,xFIP
7,2017,NYY,13,7,6,48,115.1,9.29,3.59,0.86,0.230,0.762,0.447,0.106,2.97,3.63,4.00
16,2017,LAD,15,10,5,81,135.2,8.96,2.26,1.92,0.207,0.871,0.438,0.207,3.32,4.81,3.87
25,2017,WSN,5,2,3,23,44.0,10.64,3.68,0.41,0.250,0.693,0.454,0.063,2.66,2.88,3.59
29,2017,CLE,5,2,3,28,47.0,12.26,4.02,1.34,0.264,0.763,0.418,0.179,2.68,3.77,3.31
30,2017,HOU,18,11,7,70,159.0,9.51,2.94,1.19,0.274,0.726,0.411,0.133,4.13,3.91,3.96
57,2017,MIN,1,0,1,6,8.0,5.63,5.63,3.38,0.240,0.612,0.286,0.188,9.00,8.66,7.34
61,2017,CHC,10,4,6,46,87.2,8.42,5.44,1.64,0.241,0.758,0.417,0.176,4.52,5.58,5.05
66,2017,BOS,4,1,3,20,34.0,7.94,3.71,2.12,0.376,0.766,0.293,0.160,6.35,5.95,5.51
67,2017,COL,1,0,1,8,8.0,12.38,4.50,2.25,0.536,0.549,0.414,0.222,12.38,5.16,3.91
73,2017,ARI,4,1,3,20,34.0,8.74,4.50,1.32,0.364,0.618,0.431,0.143,6.88,4.72,4.64


In [18]:
raw_2017['pitcher'] = raw_2017['pitcher'].astype(str)
rs_2017 = raw_2017.loc[raw_2017['game_type'] == 'R', :]
rs_2017 = rs_2017.loc[rs_2017['pitch_name'] != "Pitch Out", :]
ps_2017 = raw_2017.loc[(raw_2017['game_type'] == 'F') | (raw_2017['game_type'] == 'D') | (raw_2017['game_type'] == 'L') | (raw_2017['game_type'] == 'W'), :]

In [19]:
ps_2017_pitchers_by_team = {key:None for key in np.unique(np.concatenate([ps_2017['home_team'].unique(), ps_2017['away_team'].unique()]))}
for team in ps_2017_pitchers_by_team.keys():
    ps_2017_pitchers_by_team[team] = ps_2017.loc[((ps_2017['home_team'] == team) & (ps_2017['inning_topbot'] == 'Top')) | ((ps_2017['away_team'] == team) & (ps_2017['inning_topbot'] == 'Bot')), 'pitcher'].unique()

In [20]:
ps_2017_pitchers_by_team

{'AZ': array(['605177', '605151', '456696', '407822', '425844', '642073',
        '592662', '643327', '592836', '407845'], dtype=object),
 'BOS': array(['592665', '518886', '519242', '519144', '605476', '456034',
        '523260', '450729', '545348', '593958', '519141'], dtype=object),
 'CHC': array(['543557', '488846', '407793', '444468', '500779', '451584',
        '453562', '467008', '605218', '543294', '452657', '458677'],
       dtype=object),
 'CLE': array(['501925', '592102', '543766', '453192', '446372', '458708',
        '621397', '517593', '605182', '545333', '471911'], dtype=object),
 'COL': array(['518813', '608032', '459429', '450212', '543734', '542881',
        '623184', '592351'], dtype=object),
 'HOU': array(['450203', '606965', '434538', '502748', '621121', '502381',
        '605397', '434378', '501789', '543521', '572971', '571704'],
       dtype=object),
 'LAD': array(['622072', '445276', '477132', '453344', '506433', '628317',
        '453265', '448179', '548389', 

In [89]:
num = 2017
'raw' + str(num)

'raw2017'

In [ ]:
globals()['raw_' + '2017'].head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,FF,2017-11-01,96.6,-1.85,5.93,"Morton, Charlie",608369,450203,field_out,hit_into_play,NaN,NaN,NaN,NaN,14.0,"Corey Seager grounds out, second baseman Jose ...",W,L,R,LAD,HOU,X,4.0,ground_ball,0,0,2017,-0.93,0.97,1.21,2.28,NaN,NaN,NaN,2,9,Bot,159.62,145.50,NaN,NaN,NaN,NaN,10.045540,-139.883379,-6.202650,-14.184376,30.980661,-18.500614,3.45,1.56,20.0,93.7,-5.0,95.4,2097.0,5.8,526517,435263,493329,514888,608324,621043,503556,457727,543807,54.68,0.327,0.297,0.0,1.0,0.0,0.0,2.0,75,1,4-Seam Fastball,1,5,1,5,5,1,1,5,Infield shift,Standard,227.0,-0.001,-0.275,NaN,NaN,0.371,0.275,93.7,-4,-4,0.001,0.001,33,23,34,23,2,4,4.0,1.0,NaN,NaN,1.54,0.93,-0.93,NaN
1,FF,2017-11-01,95.0,-1.99,5.86,"Morton, Charlie",621035,450203,field_out,hit_into_play,NaN,NaN,NaN,NaN,13.0,"Chris Taylor grounds out softly, second basema...",W,R,R,LAD,HOU,X,4.0,ground_ball,2,2,2017,-1.20,0.81,-0.88,2.18,NaN,NaN,NaN,1,9,Bot,135.89,169.21,NaN,NaN,NaN,NaN,5.393182,-137.841463,-5.625040,-16.009884,30.165775,-21.158355,3.44,1.65,23.0,55.5,-1.0,93.5,1886.0,5.5,526517,435263,493329,514888,608324,621043,503556,457727,543807,54.97,0.087,0.099,0.0,1.0,0.0,0.0,1.0,74,5,4-Seam Fastball,1,5,1,5,5,1,1,5,Standard,Standard,240.0,-0.005,-0.241,NaN,NaN,0.099,0.241,88.0,-4,-4,0.006,0.006,33,26,34,27,2,4,4.0,1.0,NaN,NaN,1.80,1.20,1.20,NaN
2,FF,2017-11-01,96.1,-2.06,5.92,"Morton, Charlie",621035,450203,NaN,swinging_strike,NaN,NaN,NaN,NaN,5.0,"Chris Taylor grounds out softly, second basema...",W,R,R,LAD,HOU,S,NaN,NaN,2,1,2017,-1.25,1.09,0.01,2.48,NaN,NaN,NaN,1,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,8.066408,-139.307204,-5.766509,-17.615680,30.501315,-17.353954,3.48,1.70,NaN,NaN,NaN,94.6,2087.0,5.6,526517,435263,493329,514888,608324,621043,503556,457727,543807,54.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74,4,4-Seam Fastball,1,5,1,5,5,1,1,5,Standard,Standard,232.0,0.000,-0.082,NaN,NaN,NaN,0.082,NaN,-4,-4,0.006,0.006,33,26,34,27,2,4,4.0,1.0,NaN,NaN,1.47,1.25,1.25,NaN
3,FF,2017-11-01,96.7,-1.92,5.87,"Morton, Charlie",621035,450203,NaN,ball,NaN,NaN,NaN,NaN,12.0,"Chris Taylor grounds out softly, second basema...",W,R,R,LAD,HOU,B,NaN,NaN,1,1,2017,-0.99,1.33,1.13,2.67,NaN,NaN,NaN,1,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,10.217639,-139.985298,-5.846387,-15.151904,32.028166,-13.810093,3.48,1.65,NaN,NaN,NaN,95.5,2208.0,5.9,526517,435263,493329,514888,608324,621043,503556,457727,543807,54.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74,3,4-Seam Fastball,1,5,1,5,5,1,1,5,Standard,Standard,219.0,0.000,0.060,NaN,NaN,NaN,-0.060,NaN,-4,-4,0.006,0.006,33,26,34,27,2,4,4.0,1.0,NaN,NaN,1.17,0.99,0.99,NaN
4,FF,2017-11-01,96.0,-1.89,5.

In [ ]:
for team in ps_2017_pitchers_by_team.keys():
    rs_team = rs_2017.loc[np.isin(rs_2017['pitcher'], ps_pitchers_by_team[team]), ['pitcher', 'p_throws', 'pitch_name', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
    rs_team_grouped = rs_team.groupby(['pitcher'])
    rs_team_primary_pitches = rs_team_grouped['pitch_name'].agg(pd.Series.mode).to_frame()
    for pitcher in ps_2017_pitchers_by_team[team]:
        rs_primary_pitches = rs_team.loc[(rs_team['pitcher'] == pitcher) & (rs_team['pitch_name'] == rs_team_primary_pitches.loc[pitcher, :].values[0]), ['pitcher', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]

AZ
605177
605151
456696
407822
425844
642073
592662
643327
592836
407845
BOS
592665
518886
519242
519144
605476
456034
523260
450729
545348
593958
519141
CHC
543557
488846
407793
444468
500779
451584
453562
467008
605218
543294
452657
458677
CLE
501925
592102
543766
453192
446372
458708
621397
517593
605182
545333
471911
COL
518813
608032
459429
450212
543734
542881
623184
592351
HOU
450203
606965
434538
502748
621121
502381
605397
434378
501789
543521
572971
571704
LAD
622072
445276
477132
453344
506433
628317
453265
448179
548389
571561
435221
451661
MIN
279571
641427
573124
657610
621244
429722
NYY
502085
476589
592454
282332
476454
643338
622663
547888
547973
543243
448802
WSH
448281
425492
519301
445213
453286
458006
461829
544931
424144


In [ ]:
['pitcher', 'p_throws', 'pitch_type', 'pitch_name', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']

In [21]:
rs_2017_hou = rs_2017.loc[np.isin(rs_2017['pitcher'], ps_2017_pitchers_by_team['HOU']), ['pitcher', 'p_throws', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
rs_2017_hou_grouped = rs_2017_hou.groupby(['pitcher', 'p_throws'])
rs_2017_hou_grouped.agg(['mean'])

,,release_pos_x,release_pos_z,release_pos_y,release_speed,pfx_x,pfx_z,release_spin_rate,spin_axis
,,mean,mean,mean,mean,mean,mean,mean,mean
pitcher,p_throws,,,,,,,,
434378,R,-2.383328,6.388791,54.686228,91.050467,-0.576619,0.851849,2549.887440,179.920702
434538,L,0.972425,5.732391,54.322138,89.195977,0.615569,0.757161,2068.332180,160.016667
450203,R,-2.166378,5.759268,54.854367,89.602903,-0.609499,0.083450,2372.103535,184.978544
501789,R,-1.476226,6.143489,54.193519,88.431579,0.322662,0.231774,2543.451128,127.276692
502381,R,-1.602069,6.293260,54.997576,85.767607,-0.152560,0.198683,2511.435924,168.892372
502748,R,-2.088126,5.131158,54.502070,86.670748,-0.401825,0.649702,2419.388471,178.309884
543521,R,-1.410046,5.705199,54.260426,84.683133,-0.120658,0.447470,2427.999059,154.538462
571704,R,-2.062479,6.209873,55.308966,92.904852,-0.453745,0.747669,2434.347503,217.274262


In [37]:
rs_2017_hou = rs_2017.loc[np.isin(rs_2017['pitcher'], ps_2017_pitchers_by_team['HOU']), ['pitcher', 'p_throws', 'pitch_type', 'pitch_name', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
rs_2017_hou_grouped = rs_2017_hou.groupby(['pitcher'])
rs_2017_hou_primary_pitches = rs_2017_hou_grouped['pitch_name'].agg(pd.Series.mode).to_frame()

In [38]:
rs_2017_hou_primary_pitches['pitch_name'].nunique()

5

In [65]:
ps_2017_pitchers_by_team['HOU'].shape[0]

12

In [72]:
rs_2017_hou.loc[rs_2017_hou['pitcher'] == '434378', 'p_throws'].agg(pd.Series.mode)[0]

'R'

In [ ]:
rs_2017_hou_primary_pitches.index
for pitcher in ps_2017_pitchers_by_team['HOU']:
    rs_2017_hou.loc[(rs_2017_hou['pitcher'] == pitcher) & (rs_2017_hou['pitch_name'] == rs_2017_hou_primary_pitches.loc[pitcher, :].values[0]), ['pitcher', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]

Index(['434378', '434538', '450203', '501789', '502381', '502748', '543521',
       '571704', '572971', '605397', '606965', '621121'],
      dtype='object', name='pitcher')

In [73]:
series_1 = rs_2017_hou.loc[(rs_2017_hou['pitcher'] == '434378') & (rs_2017_hou['pitch_name'] == rs_2017_hou_primary_pitches.loc['434378', :].values[0]), ['release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']].agg('mean')
df_1 = series_1.to_frame().transpose()
df_1['p_throws'] = rs_2017_hou.loc[rs_2017_hou['pitcher'] == '434378', 'p_throws'].agg(pd.Series.mode)[0]

In [74]:
series_2 = rs_2017_hou.loc[(rs_2017_hou['pitcher'] == '434538') & (rs_2017_hou['pitch_name'] == rs_2017_hou_primary_pitches.loc['434538', :].values[0]), ['release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']].agg('mean')

In [87]:
test_1 = pd.concat([df_1, series_2.to_frame().transpose()]).drop(columns = ['p_throws']).agg('std').to_frame().transpose()
test_1['Season'] = 2017
test_1['Team'] = 'HOU'
test_1['n_primary_pitches'] = rs_2017_hou_primary_pitches['pitch_name'].nunique()
test_1['n_pitchers_used'] = ps_2017_pitchers_by_team['HOU'].shape[0]
pd.merge(postseason_stats, test_1, how = 'left', on = ['Season', 'Team'])

,Season,Team,TG,W,L,G,IP,K9,BB9,HR9,BABIP,LOB,GB,HRFB,ERA,FIP,xFIP,release_pos_x,release_pos_z,release_pos_y,release_speed,pfx_x,pfx_z,release_spin_rate,spin_axis,n_primary_pitches,n_pitchers_used
0,2023,PHI,13,8,5,64,114.1,9.37,2.44,0.63,0.276,0.826,0.471,0.072,2.20,2.98,3.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,HOU,13,11,2,62,126.0,11.43,3.07,0.93,0.232,0.835,0.419,0.108,2.29,3.08,3.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,MIL,10,6,4,58,93.2,10.76,2.98,0.48,0.282,0.813,0.468,0.069,2.31,2.62,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,ATL,16,11,5,80,140.0,9.96,3.21,0.84,0.293,0.781,0.420,0.096,3.28,3.41,3.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,WSN,17,12,5,66,153.0,10.24,3.35,1.35,0.275,0.817,0.363,0.135,3.47,4.09,4.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018,BOS,14,11,3,73,134.0,8.73,3.83,1.07,0.239,0.797,0.440,0.117,3.29,4.21,4.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2022,PHI,17,11,6,85,150.0,9.84,2.40,1.02,0.262,0.767,0.479,0.118,3.24,3.38,3.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,NYY,13,7,6,48,115.1,9.29,3.59,0.86,0.230,0.762,0.447,0.106,2.97,3.63,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2019,NYY,9,5,4,55,81.2,9.81,4.19,1.32,0.228,0.852,0.394,0.150,2.87,4.45,4.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2023,ARI,17,10,7,89,152.1,8.63,3.66,1.24,0.267,0.772,0.459,0.137,3.54,4.45,4.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
rs_2017_hou_primary_pitches.loc['434378', :].values[0]

'4-Seam Fastball'

In [34]:
rs_2017_hou_grouped = rs_2017_hou.groupby(['pitcher', 'p_throws', 'pitch_name'])
rs_2017_hou_grouped.agg({'pitch_type': ['count'], 'release_speed':['mean', 'std'], 
                         'release_spin_rate':['mean', 'std'], 'release_pos_x':['mean', 'std'],
                         'release_pos_z':['mean', 'std'], 'release_pos_y':['mean', 'std']})

pitch_type release_speed            \
                                      count          mean       std   
pitcher p_throws pitch_name                                           
434378  R        4-Seam Fastball       2036     95.271955  1.372827   
                 Changeup               146     87.601370  1.247618   
                 Curveball              565     80.441239  1.318855   
                 Cutter                  14     92.121429  1.633391   
                 Sinker                  12     95.641667  1.123678   
                 Slider                 758     88.191293  1.921794   
434538  L        4-Seam Fastball        294     92.783673  1.419924   
                 Changeup               294     85.429932  1.370778   
                 Curveball                9     76.355556  2.296797   
                 Sinker                 581     93.007917  1.281516   
                 Slider                 562     85.554093  2.012676   
450203  R        4-Seam Fastball        255     95.694118  0.974359   
                 Curveball              677     80.525849  1.127099   
                 Cutter                 264     88.068182  1.287041   
                 Sinker                1034     94.880948  1.012877   
                 Split-Finger           147     86.470748  0.983021   
501789  R        4-Seam Fastball         17     92.458824  1.083431   
                 Curveball              208     81.431250  0.866017   
                 Cutter                 440     91.585227  1.018792   
502381  R        4-Seam Fastball         48     89.264583  1.033171   
                 Changeup                14     80.207143  2.483318   
                 Sinker                 453     89.630022  0.835564   
                 Slider                 442     81.605430  0.938647   
502748  R        4-Seam Fastball        546     92.597802  1.021312   
                 Changeup                99     81.263636  1.920111   
                 Knuckle Curve          160     77.476875  1.520850   
                 Sinker                 607     91.776112  1.002459   
                 Sweeper                834     81.480336  1.366069   
543521  R        4-Seam Fastball        512     90.196680  1.762040   
                 Changeup                18     84.783333  1.603764   
                 Curveball              216     73.821296  1.304201   
                 Cutter                 116     86.149138  1.429898   
                 Sinker                  36     89.002778  1.239697   
                 Sweeper                181     80.240331  1.235034   
571704  R        4-Seam Fastball        500     98.166200  1.076697   
                 Changeup                 1     85.600000       NaN   
                 Slider                 447     87.036018  1.406240   
572971  L        4-Seam Fastball        119     88.830252  1.631083   
                 Changeup               283     78.687279  1.034158   
                 Cutter                 217     86.685253  1.190380   
                 Sinker                1158     88.701986  1.100446   
                 Slider                 411     78.488808  1.622065   
605397  R        4-Seam Fastball        713     93.123562  1.719740   
                 Changeup               170     84.263529  1.701746   
                 Curveball              166     81.115663  1.315454   
                 Cutter                  83     90.396386  1.629787   
                 Sinker                 143     91.645455  1.685469   
                 Slider                 484     83.207231  1.418150   
606965  R        4-Seam Fastball        504     94.024802  1.115105   
                 Changeup               480     82.983333  1.254442   
                 Slider                 285     82.843509  1.837539   
621121  R        4-Seam Fastball        491     94.211202  1.634069   
                 Changeup               230     88.709565  1.247213   
                 Knuckle Curve          952     85.647164

In [38]:
rs_2017_hou = rs_2017.loc[np.isin(rs_2017['pitcher'], ps_2017_pitchers_by_team['HOU']), ['pitcher', 'p_throws', 'pitch_name', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]
rs_2017_hou_grouped = rs_2017_hou.groupby(['pitcher', 'p_throws', 'pitch_name'])
rs_2017_hou_grouped.agg(['mean', 'std'])

release_pos_x           release_pos_z  \
                                          mean       std          mean   
pitcher p_throws pitch_name                                              
434378  R        4-Seam Fastball     -2.307932  0.179033      6.457603   
                 Changeup            -2.492945  0.136266      6.310411   
                 Curveball           -2.510248  0.142510      6.261947   
                 Cutter              -2.215714  0.670025      6.590714   
                 Sinker              -2.430833  0.147553      6.379167   
                 Slider              -2.472467  0.140997      6.310026   
434538  L        4-Seam Fastball      0.986837  0.494128      5.702959   
                 Changeup             0.727687  0.336929      5.727653   
                 Curveball            1.112222  0.600870      5.928889   
                 Sinker               1.148107  0.569256      5.661515   
                 Slider               0.909057  0.570513      5.820391   
450203  R        4-Seam Fastball     -2.077608  0.134124      5.851686   
                 Curveball           -2.224151  0.122192      5.711285   
                 Cutter              -2.237424  0.153453      5.726894   
                 Pitch Out           -2.550000  0.014142      5.870000   
                 Sinker              -2.126683  0.141907      5.779014   
                 Split-Finger        -2.205918  0.137958      5.739184   
501789  R        4-Seam Fastball     -1.455294  0.162754      6.104118   
                 Curveball           -1.481394  0.145042      6.117740   
                 Cutter              -1.474591  0.144734      6.157182   
502381  R        4-Seam Fastball     -1.409583  0.269254      6.420833   
                 Changeup            -1.858571  0.325147      6.143571   
                 Pitch Out           -1.610000  0.367696      6.385000   
                 Sinker              -1.459581  0.285730      6.382097   
                 Slider              -1.760882  0.287966      6.193100   
502748  R        4-Seam Fastball     -2.233700  0.625025      5.131612   
                 Changeup            -2.500909  0.572194      5.237374   
                 Knuckle Curve       -2.272125  0.463133      5.537125   
                 Pitch Out           -2.990000       NaN      5.380000   
                 Sinker              -1.882652  0.605028      5.033427   
                 Sweeper             -2.058070  0.625648      5.111499   
543521  R        4-Seam Fastball     -1.470098  0.161300      5.665918   
                 Changeup            -1.532222  0.105248      5.645556   
                 Curveball           -1.210972  0.176201      5.876944   
                 Cutter              -1.394310  0.182033      5.700000   
                 Sinker              -1.535278  0.140986      5.603611   
                 Sweeper             -1.450773  0.162759      5.640829   
571704  R        4-Seam Fastball     -1.988940  0.151333      6.225900   
                 Changeup            -2.280000       NaN      6.150000   
                 Slider              -2.144251  0.127093      6.192081   
572971  L        4-Seam Fastball      0.934622  0.181280      6.274286   
                 Changeup             0.998693  0.184264      6.091025   
                 Cutter               0.920323  0.182349      6.230092   
                 Sinker               1.130095  0.200922      6.153342   
                 Slider               1.031582  0.173193      6.105985   
605397  R        4-Seam Fastball     -2.020491  0.447223      6.485077   
                 Changeup            -2.221176  0.371310      6.577235   
                 Curveball           -2.019217  0.418811      6.534518   
                 Cutter              -1.683614  0.405188      6.416867   
                 Sinker              -2.118252  0.404691      6.537063   
                 Slider              -1.970103  0.463363      6.484566   
606965  R        4-Seam Fastball     

In [23]:
rs_2017_501789 = rs_2017.loc[rs_2017['pitcher'] == '501789', ['pitcher', 'p_throws', 'pitch_type', 'pitch_name', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate', 'spin_axis']]

In [24]:
rs_2017_501789_grouped = rs_2017_501789.groupby(['pitcher', 'p_throws', 'pitch_name'])

In [26]:
rs_2017_501789_grouped.agg({'pitch_type': ['count'], 'release_speed':['mean', 'std'], 
                            'release_spin_rate':['mean', 'std'], 'release_pos_x':['mean', 'std'],
                            'release_pos_z':['mean', 'std'], 'release_pos_y':['mean', 'std']})

pitch_type release_speed            \
                                      count          mean       std   
pitcher p_throws pitch_name                                           
501789  R        4-Seam Fastball         17     92.458824  1.083431   
                 Curveball              208     81.431250  0.866017   
                 Cutter                 440     91.585227  1.018792   

                                 release_spin_rate             release_pos_x  \
                                              mean         std          mean   
pitcher p_throws pitch_name                                                    
501789  R        4-Seam Fastball       2492.823529   42.770076     -1.455294   
                 Curveball             2748.754808  119.454487     -1.481394   
                 Cutter                2448.354545   88.083449     -1.474591   

                                           release_pos_z            \
                                       std          mean       std   
pitcher p_throws pitch_name                                          
501789  R        4-Seam Fastball  0.162754      6.104118  0.115166   
                 Curveball        0.145042      6.117740  0.086968   
                 Cutter           0.144734      6.157182  0.086096   

                                 release_pos_y            
                                          mean       std  
pitcher p_throws pitch_name                               
501789  R        4-Seam Fastball     54.056471  0.178008  
                 Curveball           54.385096  0.278210  
                 Cutter              54.108250  0.211046

In [75]:
rs_2017['pitch_name'].value_counts()

pitch_name
4-Seam Fastball    248944
Sinker             150403
Slider             110046
Changeup            70659
Curveball           54864
Cutter              39884
Knuckle Curve       22006
Split-Finger        12379
Sweeper              3574
Knuckleball          2646
Slurve               1953
Forkball              247
Eephus                182
Pitch Out             116
Other                  59
Screwball              39
Slow Curve              7
Name: count, dtype: int64

In [ ]:
for team in ps_2017_pitchers_by_team.keys():
    temp_df = rs_2017.loc[np.isin(rs_2017['pitcher'], ps_2017_pitchers_by_team[team]), :]
    

{'AZ': array([605177, 605151, 456696, 407822, 425844, 642073, 592662, 643327,
        592836, 407845]),
 'BOS': array([592665, 518886, 519242, 519144, 605476, 456034, 523260, 450729,
        545348, 593958, 519141]),
 'CHC': array([543557, 488846, 407793, 444468, 500779, 451584, 453562, 467008,
        605218, 543294, 452657, 458677]),
 'CLE': array([501925, 592102, 543766, 453192, 446372, 458708, 621397, 517593,
        605182, 545333, 471911]),
 'COL': array([518813, 608032, 459429, 450212, 543734, 542881, 623184, 592351]),
 'HOU': array([450203, 606965, 434538, 502748, 621121, 502381, 605397, 434378,
        501789, 543521, 572971, 571704]),
 'LAD': array([622072, 445276, 477132, 453344, 506433, 628317, 453265, 448179,
        548389, 571561, 435221, 451661]),
 'MIN': array([279571, 641427, 573124, 657610, 621244, 429722]),
 'NYY': array([502085, 476589, 592454, 282332, 476454, 643338, 622663, 547888,
        547973, 543243, 448802]),
 'WSH': array([448281, 425492, 519301, 445213, 4

In [36]:
hou_ps_2017 = ps_2017.loc[((ps_2017['home_team'] == 'HOU') & (ps_2017['inning_topbot'] == 'Top')) | ((ps_2017['away_team'] == 'HOU') & (ps_2017['inning_topbot'] == 'Bot')), :]
hou_ps_2017['pitcher'].unique()

array([450203, 606965, 434538, 502748, 621121, 502381, 605397, 434378,
       501789, 543521, 572971, 571704])